In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [3]:
with open("example.txt", "r") as file:
    content = file.read()


In [4]:
examples = content.strip().split("Example:")

In [5]:
post_data = []
comment_data = []

In [6]:
for i in range(1, 11):
    post_data.append(examples[i].strip().split('\n\n')[0])
    comment_data.append(examples[i].split('\n\n')[1])
    comment_data.append(examples[i].split('\n\n')[2])
    comment_data.append(examples[i].split('\n\n')[3])
    

In [7]:
post_columns = ["Post ID", "Title", "Content", "Timestamp", "Author ID", "Upvote Ratio", "Number of Comments", "Post Type"]
comment_columns = ["Comment ID", "Post ID", "Content", "Timestamp", "Author ID", "Upvotes"]

post_df = pd.DataFrame([post.split('\n') for post in post_data], columns=post_columns)
comment_df = pd.DataFrame([comment.split('\n') for comment in comment_data], columns=comment_columns)

In [8]:
print("Posts DataFrame:")
print(post_df[:5])
print("\nComments DataFrame:")
print(comment_df[:5])

Posts DataFrame:
  Post ID                                     Title  \
0   p1xrp               Ripple's Recent Price Surge   
1   p2xrp  Technical Analysis of XRP Price Movement   
2   p3xrp            XRP Adoption Continues to Grow   
3   p4xrp  Regulatory Developments Impact XRP Price   
4   p5xrp        XRP Community Celebrates Milestone   

                                             Content            Timestamp  \
0  Ripple (XRP) has experienced a significant pri...  2023-08-15 09:30:00   
1  Analyzing the technical indicators, it seems t...  2023-08-15 14:15:00   
2  More businesses are integrating XRP as a payme...  2023-08-16 11:00:00   
3  Recent regulatory announcements have caused fl...  2023-08-17 16:45:00   
4  The XRP community has reached a milestone of 1...  2023-08-18 08:00:00   

             Author ID Upvote Ratio Number of Comments    Post Type  
0      cryptoTrader123         0.92                 25   News        
1          TAexpert456         0.85              

In [15]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
